# Bitcoin Mechanisms

In this project, different mechanisms of Bitcoin are interacted with and simulated.

# Table Of Contents
[1] Address Generation
[2] Transaction Submission and Execution
[3] Mining
[4] References

# Address Generation
In this section, we generate a new address on Bitcoin testnet.

## Generating a Base58 Bitcoin Testnet Address

To generate a Base58 Bitcoin testnet address, we need to first generate a private key and then use it to derive the corresponding public key and address. The private key is a random 256-bit number that is used to sign Bitcoin transactions, while the public key is derived from the private key and is used to verify the signature. The address is a Base58-encoded representation of the public key.

It is important to note that Bitcoin testnet addresses have a different prefix than mainnet addresses to prevent confusion between the two networks. Testnet addresses start with the letter "m" or "n," while mainnet addresses start with the number "1" or "3."

Differences Between Mainnet and Testnet Addresses

The main difference between mainnet and testnet addresses is that testnet addresses are used for testing and development purposes only. Transactions on the testnet do not involve real bitcoins and are not recorded on the mainnet blockchain. Testnet addresses use a simpler algorithm for generating private keys, which makes them easier to work with during testing and development. In contrast, mainnet addresses use a more complex algorithm to generate private keys, which provides a higher level of security against hacking and theft.

## Method for Generating a Base58 Bitcoin Testnet Address using Python

To generate a Base58 Bitcoin testnet address in Python, we can use the following steps:

1. Generate a random 256-bit number to use as the private key.
2. Derive the public key from the private key using the Elliptic Curve Digital Signature Algorithm (ECDSA).
3. Apply SHA-256 and RIPEMD160 hashing algorithms to the public key to generate a hash.
4. Add a version byte prefix to the hash. For testnet addresses, this is 0x6f.
5. Apply a checksum to the version byte prefix and hash using the SHA-256 hashing algorithm.
6. Concatenate the version byte prefix, hash, and checksum to form the raw address.
7. Encode the raw address using Base58 encoding to obtain the final testnet address.

## Generating the Private Key

In [13]:
import base58
import random
import hashlib

In [14]:
def checksum(key):
    return hashlib.sha256(hashlib.sha256(key).digest()).digest()[:4]


def to_wif(key):
    # The version byte prefix (0xEF for Bitcoin testnet, 0x00 for mainnet)
    version_byte_prefix = b'\xef'

    extended_key = version_byte_prefix + bytes.fromhex(key)

    # Calculate the checksum by taking the first 4 bytes of the double SHA-256 hash
    # of the extended key and add the checksum to the extended key
    extended_key_with_checksum = extended_key + checksum(extended_key)

    return base58.b58encode(extended_key_with_checksum).decode()

In [15]:
private_key = hex(random.getrandbits(256))
# private_key = "0xa960f9ba176535a72b0c0bda6df6eda708fd8435a8c8b4fa8239efbd3cac1f1b"
# private_key = "0x804c96c0753893a31d3d09a1475ac3b1b8512799f66f0533413014bf91a6041d"
wif = to_wif(private_key[2:])

# Verify using: https://learnmeabitcoin.com/technical/wif
print("Private Key Hex:", private_key)
print("Testnet WIF:", wif)

Private Key Hex: 0x8da6b5b487ee2a9ff197b99653ff5c4f83b313bad1bb5d3f788d71d16e703f2a
Testnet WIF: 92fJTEz692SMTKCETXR1nri4c5aKjeLvgMtUJfaaYoAxTpm9odG


## Deriving Public Key from Private Key

In [16]:
def e_add(_x, _y, Gx, Gy, p):
    if _x == 0 and _y == 0:
        return Gx, Gy

    if Gx == 0 and Gy == 0:
        return _x, _y

    if _x == Gx and _y == Gy:
        delta = pow(3 * pow(_x, 2, p) * pow(2 * _y, -1, p), 1, p)
    elif _x == Gx:
        return 0, 0
    else:
        delta = pow(pow(Gy - _y, 1, p) * pow(Gx - _x, -1, p), 1, p)

    new_x = pow(pow(delta, 2) - Gx - _x, 1, p)
    new_y = pow(delta * (_x - new_x) - _y, 1, p)
    return new_x, new_y


def secp256k1(_private_key, _result_arr):
    p = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
    _x, _y = (0, 0)
    for i in range(len(bin(_private_key)[2:])):
        if bin(_private_key)[:1:-1][i] == '1':
            _x, _y = e_add(_x, _y, _result_arr[i][0], _result_arr[i][1], p)
    return _x, _y


def generate_G_coeffs():
    p = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
    _x = int(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798)
    _y = int(0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8)
    result_arr = [(0, 0)] * 256
    for i in range(256):
        result_arr[i] = (_x, _y)
        _x, _y = e_add(_x, _y, _x, _y, p)
    return result_arr

In [17]:
x, y = secp256k1(int(private_key[2:], 16), generate_G_coeffs())
hex_x = hex(x)[2:]
hex_y = hex(y)[2:]
public_key = '04' + hex_x + hex_y
compressed_public_key = '03' + hex_x if int(hex(y), 16) % 2 == 1 else '02' + hex_x

# For validation check: https://learnmeabitcoin.com/technical/public-key
print('Public key : ', public_key)
print('Compressed public key : ', compressed_public_key)

Public key :  04f4d55f6fc722f1607c5e93e1a2c52f3da9b7716f90309179ee112cdfd830cf7d7cd7ef6c5f694aaf03e94eec01ef24c3a3fc94095c955074b08bcd0cfe8a642f
Compressed public key :  03f4d55f6fc722f1607c5e93e1a2c52f3da9b7716f90309179ee112cdfd830cf7d


## Generating the Address

In [18]:
def generate_address(_public_key):
    # P2PKH Address (6f for testnet and 00 for mainnet)
    version_byte_prefix = b'\x6f'

    # Hash the given public key
    public_key_bytes = bytes.fromhex(_public_key)
    sha256_1 = hashlib.sha256(public_key_bytes).digest()
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha256_1)
    hash160 = ripemd160.digest()

    # Add version byte to the beginning of the RIPEMD-160 hash
    hash160_with_version = version_byte_prefix + hash160

    sha256_2 = hashlib.sha256(hashlib.sha256(hash160_with_version).digest()).digest()

    hash160_with_checksum = hash160_with_version + sha256_2[:4]  #(checksum)

    return base58.b58encode(hash160_with_checksum).decode()

In [19]:
# Verify using: https://learnmeabitcoin.com/technical/address
address = generate_address(public_key)
print('Address: ', address)
compressed_address = generate_address(compressed_public_key)
print('Compressed Address: ', compressed_address)

Address:  mgX6n5gJcaD216N1zM4Hj76wApTs3LopjA
Compressed Address:  n4R21kv2NtGLzkmuVjFfsZ4PEx2jXZ7Hqs


## Generating Vanity Address

In [20]:
def generate_vanity_address(initial):
    result_arr = generate_G_coeffs()
    while True:
        _vanity_private_key = hex(random.getrandbits(256))
        _x, _y = secp256k1(int(_vanity_private_key[2:], 16), result_arr)

        # Zero-pad _x to a fixed width of 64 hexadecimal digits (32 bytes)
        _x_hex = '{:0>64x}'.format(_x)

        _vanity_public_key = '03' + _x_hex if int(hex(_y), 16) % 2 == 1 else '02' + _x_hex
        _vanity_address = generate_address(_vanity_public_key)
        print(_vanity_address)
        if _vanity_address[1:4] == initial:
            return _vanity_address, _vanity_public_key, _vanity_private_key

In [ ]:
vanity_address, vanity_public_key, vanity_private_key = generate_vanity_address('Jee')
print('vanity address : %s' % vanity_address)
print(vanity_public_key)
wifVanity = to_wif(vanity_private_key[2:])
print('wif : %s' % wifVanity)

mjywCMCfwDqvgXUzo3HqQ2NM8hmHLJyTpe
mos3yYY7c4RMZFMWGGLtos7Hdx1FyZ7bK7
n2gFUmL7CvnwFGDhSrNyyPp4k692TzWhxZ
n3MhjSdCB5DbjFU5J7LW4y8HkkwTpRKiu7
mzakVYoDLZehLza4KCnHd663FXbtizmF2Z
n1bNqJmZSZZyaEo55W43ZxGJ1kBR8xTkiP
myyfTLAEW7i2hzof8Y5Pb8f6DY4RuECHUD
moMhGREMgyQatRCRsXmc8HGE7zbAXcsZLa
mzySJycxBAtgqcQaBi1mP8xsPRa5RqvHx9
mpkwsF2DuKtPGfJ9rNB3jWnCc6a2EwWk4R
mswnXZKqfoxfJCCSuR9ksJyFxFMKtExKPF
mx4Ni3QKnHyT4A8AUj5SyPrDoRg65jM6s3
mh4vLzTBEAiBQPHDh9cen5fByeRUfjYidG
msNtLWhSVrei6KPQyJTZn6CkBmbcwT4wym
mp1gWQBEZj88e68FX321i6kWgupvYS75HN
mpBWPwUJMMbtELemoosHsjNsZQzwgpXr8p
n2nJVDaYqCtyXzbDjhw4ReN5eHad9aXyhj
mxCDP3xTVh2L5AFC3MJW5WuHE2xCEgDqcY
n1T6RYySEFTngZ41dCLS9V1oyxZAqmETwS
mxyuMcGZudJMZfQVMnnYjEWQxx9cWCPSMg
mgUxjVgJLtPLPJgM93bzcewUotCsJSC2S5
n3bGqGuYBcKQHzUTmtzKf72JRtzwTETtJt
mkcNbw3zqBqNhS82adgdD5T6CCEw2cfK11
mqfjWwiULHL1AB76qee5DufHfkKmWGqovd
mkxLVENSq5fEEr1YVqHaPXEkmJhSnmDYEM
my3ZbvtMm1RZJh9ueZvuGZ7TmoAC2DHPoQ
mgxkRyY1px2AoFtej1ptZ25erMArS8ERLd
mrcjMEzmQrNUXtewHcWdhvvedrpoEFRev5
msTvfor4dqo3xRsSV7XA

In [11]:
for i in range(4):
    print(i)
    private_key = hex(random.getrandbits(256))
    wif = to_wif(private_key[2:])
    x, y = secp256k1(int(private_key[2:], 16), generate_G_coeffs())
    hex_x = hex(x)[2:]
    hex_y = hex(y)[2:]
    public_key = '04' + hex_x + hex_y
    address = generate_address(public_key)
    print('PrvKey: ', private_key)
    print('WIF: ', wif)
    print('Address: ', address)

0
PrvKey:  0x78c5b816a13afbe2772dc3249731b3ab54a2f72b98d7ba731d578d686efdc1ad
WIF:  92W78XPjA2fTR4qGcFQed5bAv19mJZkzDkXRLHyZob3DBRMgiPU
Address:  miUse5MqoUGGFiKEFaneGygqGVPeh6MDuL
1
PrvKey:  0x7d50073513208ab2806fc875132f3f6d7486abaf154765de85ce42292f28e9e4
WIF:  92Y775cuvvvdjsoiRdu3ntEV3d22Zk1AR1HvqR2DDU8HCW1xkRo
Address:  miqigBBdvJnHAg9P1VRB8c8gisCsmuEZRM
2
PrvKey:  0xf6f60ebd9a65aeee4bde8a69c5d485f7c21ba5d0261e9d2019ad45dd21c129d7
WIF:  93TgTKfXUyTEcGXsWjuwptBs9XgoXrfb1vw6AJpi68a3eQbz3aa
Address:  myNPS4SnNojLDRtrY95kWHDSRHgapqTxTs
3
PrvKey:  0x1d631e1d20f6bb41e48a6f6a0a7d856386cdcf17bba6e26422310fc2344baa14
WIF:  91orpU9JPHAkCzg6RdQcFpowHQGakPk4YZb3Yv9QFNrpwaTbUx2
Address:  myM3e2ZE7fDjY4ZKGxDbnpMsJJWv5uzEvT
